In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hypermri.utils.utils_fitting as utf
import hypermri.utils.utils_spectroscopy as uts
import hypermri.utils.utils_general as utg
from tqdm.auto import tqdm
import ipywidgets as widgets
import matplotlib
import os
import sys
matplotlib.rcParams.update({'font.size': 12,'font.family':'serif','font.serif':['Computer Modern'],"text.usetex" : True,})
# Get the current working directory
cwd = os.getcwd()
parent_dir = os.path.dirname(os.path.dirname(cwd))
sys.path.append(parent_dir)
# define paths:
sys.path.append('../')
import TEMPLATE
# get paths:
repopath, basepath, savepath,publication_path = TEMPLATE.import_all_packages(True)
%matplotlib widget

# 1. SNR simulation

In [ ]:
bandwidth = 5000
dwell_time = 1/bandwidth
samplings = [2048]
repeats = 100
noise_base_level=0.01
b0=3
fieldoffset=170.4
noise_levels=np.linspace(noise_base_level,1,50)

In [ ]:
temps_truth = np.zeros((len(noise_levels),repeats,len(samplings)))
temps_fit = np.zeros((len(noise_levels),repeats,len(samplings)))
pyr_lac_diffs_fit = np.zeros((len(noise_levels),repeats,len(samplings)))
pyr_lac_diffs_fit_ppm= np.zeros((len(noise_levels),repeats,len(samplings)))
pyr_lac_diffs_truth = np.zeros((len(noise_levels),repeats,len(samplings)))
resolutions = np.zeros((len(noise_levels),repeats,len(samplings)))
snrs = np.zeros((len(noise_levels),repeats,len(samplings)))

for sampling_index, sampling in tqdm(enumerate(samplings),total=len(samplings),desc='Sampling'):
    input_data = np.zeros((sampling,len(noise_levels),repeats,1,1,1),dtype=complex)
    for noise_level_index,noise_level in enumerate(noise_levels):
        for repeat_index in range(repeats):
            metab_fs = np.array([19.0,410.0,-313.0,273.0]) #Hz


            time = np.linspace(0, dwell_time * (sampling - 1), sampling)
            signal_temp =  uts.generate_fid(amplitude=[1.2, 0.85, 0.3, 0.3],
                                       T2_s=[0.05, 0.05, 0.05, 0.05],
                                       freq0_Hz=metab_fs,
                                       sampling_dt=dwell_time,
                                       npoints=sampling,
                                       noise_amplitude=noise_level,
                                       sum_fids=True) 
            input_data[:,noise_level_index,repeat_index,0,0,0]=signal_temp
            freq_range = np.fft.fftshift(np.fft.fftfreq(sampling,dwell_time))
            ppm_range = freq_range/(utg.get_gmr(nucleus="13c")*b0) + fieldoffset
            spectrum=np.fft.fftshift(np.fft.fft(signal_temp))
            pyr_peak_height = np.max(np.abs(spectrum))
            std = np.std(np.abs(spectrum[:50]))
            mean=np.mean(np.abs(spectrum[:50]))
            snr = (pyr_peak_height-mean)/std
            snrs[noise_level_index,repeat_index,sampling_index]=snr
    metabs = ['pyruvate', 'lactate', 'bicarbonate', 'pyruvatehydrate']
    niter = 2 # number of iterations:
    npoints  = 31# number of tested points per iteration:
    rep_fitting = 11
    # define fit parameters:
    fit_params = {}
    # define peak frequencies:
    fit_params["metabs"] = metabs
    fit_params["signal_domain"] = "temporal"
    fit_params["b0"] = b0
    fit_params["init_fit"] = True
    fit_params["coff"] = 0
    fit_params["zoomfactor"] = 1.5
    fit_params["metab_t2s"] = [0.05 for _ in metabs]
    fit_params["max_t2_s"] = [0.07, 0.07, 0.07, 0.07]
    fit_params["min_t2_s"] = [0.015, 0.03, 0.03, 0.03]
    fit_params["range_t2s_s"] = 0.05
    gmr = utg.get_gmr(nucleus="13c")
    fit_params["freq_range_Hz"] = freq_range
    fit_params["freq_range_ppm"] = fit_params["freq_range_Hz"]/(gmr*fit_params['b0']) + fieldoffset
    fit_params["range_freqs_Hz"] = 20
    fit_params["show_tqdm"] = True
    fit_params["metabs_freqs_ppm"] = [uts.get_metab_cs_ppm(metab=m) for m in metabs]
    # define peak frequencies:
    # TODO: implement maybe better finding of spectrum to get peak pos from
    spectrum = np.fft.fftshift(np.fft.fft(input_data[:,0,0,0,0,0]))
    max_peakindex = uts.find_npeaks(input_data=spectrum,
                                    npeaks=1,
                                    freq_range=fit_params['freq_range_Hz'],
                                    plot=False,
                                   sort_order=None)
    pyr_data =  fit_params["freq_range_ppm"][max_peakindex][0]
    pyr_lit = fit_params["metabs_freqs_ppm"][0]
    diff_pyr_data_lit = pyr_data - pyr_lit
    fit_params["metabs_freqs_ppm"] = [uts.get_metab_cs_ppm(metab=m)+diff_pyr_data_lit for m in metabs]

    fit_params["niter"] = niter # number of iterations:
    fit_params["npoints"] = npoints # number of tested points per iteration:
    fit_params["rep_fitting"] = rep_fitting # number of tested points per iteration:

    fit_params["fit_range_repetitions"] = 1
    fit_params["use_all_cores"]=True
    fit_params = utf.def_fit_params(fit_params=fit_params)
    fit_spectrums, fit_amps, fit_freqs, fit_t2s,  fit_stds = utf.fit_data_pseudo_inv(input_data=input_data,
                                                                      fit_params=fit_params, 
                                                                      dbplot=False,
                                                                      use_multiprocessing=True)

    # sorting back the results
    for noise_level_index,noise_level in enumerate(noise_levels):
        for repeat_index in range(repeats):
            metab_fs = np.array([19.0,410.0,-313.0,273.0]) #Hz

            freq_range = np.fft.fftshift(np.fft.fftfreq(sampling,dwell_time))
            ppm_range = freq_range/(utg.get_gmr(nucleus="13c")*b0) + fieldoffset
            spectral_resolution=bandwidth/sampling
            pyr_lac_diff_fit = fit_freqs[0,noise_level_index,repeat_index,0,0,0,1]-fit_freqs[0,noise_level_index,repeat_index,0,0,0,0]
            pyr_lac_diff_ppm_fit = uts.freq_Hz_to_ppm(pyr_lac_diff_fit,freq_range,ppm_range,ppm_centered_at_0=True)
            temp_fit,_=utf.temperature_from_frequency(pyr_lac_diff_ppm_fit,'5mM',True)

            pyr_lac_diff_truth=metab_fs[1]-metab_fs[0]
            pyr_lac_diff_ppm_truth=uts.freq_Hz_to_ppm(pyr_lac_diff_truth,freq_range,ppm_range,ppm_centered_at_0=True)
            temp_truth,_=utf.temperature_from_frequency(pyr_lac_diff_ppm_truth,'5mM',True)

            temps_truth[noise_level_index,repeat_index,sampling_index]=temp_truth
            temps_fit[noise_level_index,repeat_index,sampling_index]=temp_fit
            pyr_lac_diffs_fit[noise_level_index,repeat_index,sampling_index]=pyr_lac_diff_fit
            pyr_lac_diffs_fit_ppm[noise_level_index,repeat_index,sampling_index]=pyr_lac_diff_ppm_fit
            
            pyr_lac_diffs_truth[noise_level_index,repeat_index,sampling_index]=pyr_lac_diff_truth
            resolutions[noise_level_index,repeat_index,sampling_index]=spectral_resolution

In [ ]:
data = dict()
data.update({'temps_truth':temps_truth,
                   'temps_fit':temps_fit,
                   'pyr_lac_diffs_fit':pyr_lac_diffs_fit,
                   'pyr_lac_diffs_truth':pyr_lac_diffs_truth,
                   'noise_levels':noise_levels,
                   'samplings':samplings,
                   'resolutions':resolutions,
             'repeats':repeats,
            'snrs':snrs})
utg.save_as_pkl('','sim_SNR_more_vals_rev1',data)

# 2. Linebroadening simulation

In [ ]:
def apply_lb(fid,time_ax,lb):
    return fid*np.exp(-np.pi*lb*time_ax)

In [ ]:
bandwidth = 5000
dwell_time = 1/bandwidth
linebroadenings=np.linspace(0,100,50)
samplings = [2048]
noise_base_level=0.01
repeats=100



temps_truth = np.zeros((len(linebroadenings),repeats,len(samplings)))
temps_fit = np.zeros((len(linebroadenings),repeats,len(samplings)))
pyr_lac_diffs_fit = np.zeros((len(linebroadenings),repeats,len(samplings)))
pyr_lac_diffs_truth = np.zeros((len(linebroadenings),repeats,len(samplings)))
resolutions = np.zeros((len(linebroadenings),repeats,len(samplings)))
snrs = np.zeros((len(linebroadenings),repeats,len(samplings)))

for sampling_index, sampling in tqdm(enumerate(samplings),total=len(samplings),desc='Sampling'):
    input_data = np.zeros((sampling,len(linebroadenings),repeats,1,1,1),dtype=complex)
    for lb_index,lb_val in enumerate(linebroadenings):
        for repeat_index in range(repeats):
            metab_fs = np.array([19.0,410.0,-313.0,273.0]) #Hz


            time = np.linspace(0, dwell_time * (sampling - 1), sampling)
            signal_temp =  uts.generate_fid(amplitude=[1.2, 0.85, 0.3, 0.3],
                                       T2_s=[0.05, 0.05, 0.05, 0.05],
                                       freq0_Hz=metab_fs,
                                       sampling_dt=dwell_time,
                                       npoints=sampling,
                                       noise_amplitude=noise_base_level,
                                       sum_fids=True) 
            #apply linebroadening
            signal_temp=apply_lb(signal_temp,time,lb_val)
            
            input_data[:,lb_index,repeat_index,0,0,0]=signal_temp
            freq_range = np.fft.fftshift(np.fft.fftfreq(sampling,dwell_time))
            ppm_range = freq_range/(utg.get_gmr(nucleus="13c")*3) + 175
            spectrum=np.fft.fftshift(np.fft.fft(signal_temp))
            pyr_peak_height = np.max(np.abs(spectrum))
            std = np.std(np.real(spectrum[:50]))
            mean = np.mean(np.real(spectrum[:50]))
            
            snr = (pyr_peak_height-mean)/std
            snrs[lb_index,repeat_index,sampling_index]=snr
    metabs = ['pyruvate', 'lactate', 'bicarbonate', 'pyruvatehydrate']
    niter = 2 # number of iterations:
    npoints  = 31# number of tested points per iteration:
    rep_fitting = 11
    # define fit parameters:
    fit_params = {}
    # define peak frequencies:
    fit_params["metabs"] = metabs
    fit_params["signal_domain"] = "temporal"
    fit_params["b0"] = 3
    fit_params["init_fit"] = True
    fit_params["coff"] = 0
    fit_params["zoomfactor"] = 1.5
    fit_params["metab_t2s"] = [0.05 for _ in metabs]
    fit_params["max_t2_s"] = [0.07, 0.07, 0.07, 0.07]
    fit_params["min_t2_s"] = [0.015, 0.03, 0.03, 0.03]
    fit_params["range_t2s_s"] = 0.05
    gmr = utg.get_gmr(nucleus="13c")
    fit_params["freq_range_Hz"] = freq_range
    fit_params["freq_range_ppm"] = fit_params["freq_range_Hz"]/(gmr*fit_params['b0']) + 175
    fit_params["range_freqs_Hz"] = 20
    fit_params["show_tqdm"] = True
    fit_params["metabs_freqs_ppm"] = [uts.get_metab_cs_ppm(metab=m) for m in metabs]
    # define peak frequencies:
    # TODO: implement maybe better finding of spectrum to get peak pos from
    spectrum = np.fft.fftshift(np.fft.fft(input_data[:,0,0,0,0,0]))
    max_peakindex = uts.find_npeaks(input_data=spectrum,
                                    npeaks=4,
                                    freq_range=fit_params['freq_range_ppm'],
                                    plot=False,
                                   sort_order=None)
    pyr_data =  fit_params["freq_range_ppm"][max_peakindex][0]
    pyr_lit = fit_params["metabs_freqs_ppm"][0]
    diff_pyr_data_lit = pyr_data - pyr_lit
    fit_params["metabs_freqs_ppm"] = [uts.get_metab_cs_ppm(metab=m)+diff_pyr_data_lit for m in metabs]

    fit_params["niter"] = niter # number of iterations:
    fit_params["npoints"] = npoints # number of tested points per iteration:
    fit_params["rep_fitting"] = rep_fitting # number of tested points per iteration:

    fit_params["fit_range_repetitions"] = 1
    fit_params["use_all_cores"]=True
    fit_params = utf.def_fit_params(fit_params=fit_params)
    fit_spectrums, fit_amps, fit_freqs, fit_t2s,  fit_stds = utf.fit_data_pseudo_inv(input_data=input_data,
                                                                      fit_params=fit_params, 
                                                                      dbplot=False,
                                                                      use_multiprocessing=True)

    # sorting back the results
    for lb_index,lb_value in enumerate(linebroadenings):
        for repeat_index in range(repeats):
            metab_fs = np.array([19.0,410.0,-313.0,273.0]) #Hz

            freq_range = np.fft.fftshift(np.fft.fftfreq(sampling,dwell_time))
            ppm_range = freq_range/(utg.get_gmr(nucleus="13c")*3) + 175
            spectral_resolution=bandwidth/sampling
            pyr_lac_diff_fit = fit_freqs[0,lb_index,repeat_index,0,0,0,1]-fit_freqs[0,lb_index,repeat_index,0,0,0,0]
            pyr_lac_diff_ppm_fit = uts.freq_Hz_to_ppm(pyr_lac_diff_fit,freq_range,ppm_range,ppm_centered_at_0=True)
            temp_fit,_=utf.temperature_from_frequency(pyr_lac_diff_ppm_fit,'5mM',True)

            pyr_lac_diff_truth=metab_fs[1]-metab_fs[0]
            pyr_lac_diff_ppm_truth=uts.freq_Hz_to_ppm(pyr_lac_diff_truth,freq_range,ppm_range,ppm_centered_at_0=True)
            temp_truth,_=utf.temperature_from_frequency(pyr_lac_diff_ppm_truth,'5mM',True)

            temps_truth[lb_index,repeat_index,sampling_index]=temp_truth
            temps_fit[lb_index,repeat_index,sampling_index]=temp_fit
            pyr_lac_diffs_fit[lb_index,repeat_index,sampling_index]=pyr_lac_diff_fit
            pyr_lac_diffs_truth[lb_index,repeat_index,sampling_index]=pyr_lac_diff_truth
            resolutions[lb_index,repeat_index,sampling_index]=spectral_resolution

In [ ]:
temps_fit.shape

In [ ]:
data = dict()
data.update({'temps_truth':temps_truth,
                   'temps_fit':temps_fit,
                   'pyr_lac_diffs_fit':pyr_lac_diffs_fit,
                   'pyr_lac_diffs_truth':pyr_lac_diffs_truth,
                   'linebroadenings':linebroadenings,
                   'repeats':repeats,
                   'samplings':samplings,
                   'resolutions':resolutions,
             'noise_base_level':noise_base_level,
             'bandwidth':bandwidth,
            'snrs':snrs})
utg.save_as_pkl('','sim_varying_lb',data)

# 3. T2* simulation

In [ ]:
bandwidth = 5000
dwell_time = 1/bandwidth
samplings = [2048]
repeats = 100
noise_base_level=0.01
b0=3
fieldoffset=170.4

fwhms=np.arange(5,150,2)
t2s=1/(np.pi*fwhms)


In [ ]:
temps_truth = np.zeros((len(t2s),repeats,len(samplings)))
temps_fit = np.zeros((len(t2s),repeats,len(samplings)))
pyr_lac_diffs_fit = np.zeros((len(t2s),repeats,len(samplings)))
pyr_lac_diffs_fit_ppm= np.zeros((len(t2s),repeats,len(samplings)))
pyr_lac_diffs_truth = np.zeros((len(t2s),repeats,len(samplings)))
resolutions = np.zeros((len(t2s),repeats,len(samplings)))
snrs = np.zeros((len(t2s),repeats,len(samplings)))

for sampling_index, sampling in tqdm(enumerate(samplings),total=len(samplings),desc='Sampling'):
    input_data = np.zeros((sampling,len(t2s),repeats,1,1,1),dtype=complex)
    for t2_index,t2_val in enumerate(t2s):
        for repeat_index in range(repeats):
            metab_fs = np.array([19.0,410.0,-313.0,273.0]) #Hz


            time = np.linspace(0, dwell_time * (sampling - 1), sampling)
            signal_temp =  uts.generate_fid(amplitude=[1.2, 0.85, 0.3, 0.3],
                                       T2_s=np.array([1,1,1,1])*t2_val,
                                       freq0_Hz=metab_fs,
                                       sampling_dt=dwell_time,
                                       npoints=sampling,
                                       noise_amplitude=noise_base_level,
                                       sum_fids=True) 
            input_data[:,t2_index,repeat_index,0,0,0]=signal_temp
            freq_range = np.fft.fftshift(np.fft.fftfreq(sampling,dwell_time))
            ppm_range = freq_range/(utg.get_gmr(nucleus="13c")*b0) + fieldoffset
            spectrum=np.fft.fftshift(np.fft.fft(signal_temp))
            pyr_peak_height = np.max(np.abs(spectrum))
            std = np.std(np.abs(spectrum[:50]))
            mean=np.mean(np.abs(spectrum[:50]))
            snr = (pyr_peak_height-mean)/std
            snrs[t2_index,repeat_index,sampling_index]=snr
    metabs = ['pyruvate', 'lactate', 'bicarbonate', 'pyruvatehydrate']
    niter = 2 # number of iterations:
    npoints  = 31# number of tested points per iteration:
    rep_fitting = 11
    # define fit parameters:
    fit_params = {}
    # define peak frequencies:
    fit_params["metabs"] = metabs
    fit_params["signal_domain"] = "temporal"
    fit_params["b0"] = b0
    fit_params["init_fit"] = True
    fit_params["coff"] = 0
    fit_params["zoomfactor"] = 1.5
    fit_params["metab_t2s"] = [0.05 for _ in metabs]
    fit_params["max_t2_s"] = [0.13, 0.13, 0.13, 0.13]
    fit_params["min_t2_s"] = [0.001,0.001,0.001,0.001]

    gmr = utg.get_gmr(nucleus="13c")
    fit_params["freq_range_Hz"] = freq_range
    fit_params["freq_range_ppm"] = fit_params["freq_range_Hz"]/(gmr*fit_params['b0']) + fieldoffset
    fit_params["range_freqs_Hz"] = 50
    fit_params["show_tqdm"] = True
    fit_params["metabs_freqs_ppm"] = [uts.get_metab_cs_ppm(metab=m) for m in metabs]
    # define peak frequencies:
    # TODO: implement maybe better finding of spectrum to get peak pos from
    spectrum = np.fft.fftshift(np.fft.fft(input_data[:,0,0,0,0,0]))
    max_peakindex = uts.find_npeaks(input_data=spectrum,
                                    npeaks=1,
                                    freq_range=fit_params['freq_range_Hz'],
                                    plot=False,
                                   sort_order=None)
    pyr_data =  fit_params["freq_range_ppm"][max_peakindex][0]
    pyr_lit = fit_params["metabs_freqs_ppm"][0]
    diff_pyr_data_lit = pyr_data - pyr_lit
    fit_params["metabs_freqs_ppm"] = [uts.get_metab_cs_ppm(metab=m)+diff_pyr_data_lit for m in metabs]

    fit_params["niter"] = niter # number of iterations:
    fit_params["npoints"] = npoints # number of tested points per iteration:
    fit_params["rep_fitting"] = rep_fitting # number of tested points per iteration:

    fit_params["fit_range_repetitions"] = 1
    fit_params["use_all_cores"]=True
    fit_params = utf.def_fit_params(fit_params=fit_params)
    fit_spectrums, fit_amps, fit_freqs, fit_t2s,  fit_stds = utf.fit_data_pseudo_inv(input_data=input_data,
                                                                      fit_params=fit_params, 
                                                                      dbplot=False,
                                                                      use_multiprocessing=True)

    # sorting back the results
    for t2_index,t2_val in enumerate(t2s):
        for repeat_index in range(repeats):
            metab_fs = np.array([19.0,410.0,-313.0,273.0]) #Hz

            freq_range = np.fft.fftshift(np.fft.fftfreq(sampling,dwell_time))
            ppm_range = freq_range/(utg.get_gmr(nucleus="13c")*b0) + fieldoffset
            spectral_resolution=bandwidth/sampling
            pyr_lac_diff_fit = fit_freqs[0,t2_index,repeat_index,0,0,0,1]-fit_freqs[0,t2_index,repeat_index,0,0,0,0]
            pyr_lac_diff_ppm_fit = uts.freq_Hz_to_ppm(pyr_lac_diff_fit,freq_range,ppm_range,ppm_centered_at_0=True)
            temp_fit,_=utf.temperature_from_frequency(pyr_lac_diff_ppm_fit,'5mM',True)

            pyr_lac_diff_truth=metab_fs[1]-metab_fs[0]
            pyr_lac_diff_ppm_truth=uts.freq_Hz_to_ppm(pyr_lac_diff_truth,freq_range,ppm_range,ppm_centered_at_0=True)
            temp_truth,_=utf.temperature_from_frequency(pyr_lac_diff_ppm_truth,'5mM',True)

            temps_truth[t2_index,repeat_index,sampling_index]=temp_truth
            temps_fit[t2_index,repeat_index,sampling_index]=temp_fit
            pyr_lac_diffs_fit[t2_index,repeat_index,sampling_index]=pyr_lac_diff_fit
            pyr_lac_diffs_fit_ppm[t2_index,repeat_index,sampling_index]=pyr_lac_diff_ppm_fit
            
            pyr_lac_diffs_truth[t2_index,repeat_index,sampling_index]=pyr_lac_diff_truth
            resolutions[t2_index,repeat_index,sampling_index]=spectral_resolution

In [ ]:
data = dict()
data.update({'temps_truth':temps_truth,
                   'temps_fit':temps_fit,
                   'pyr_lac_diffs_fit':pyr_lac_diffs_fit,
                   'pyr_lac_diffs_truth':pyr_lac_diffs_truth,
                   't2s':t2s,
                   'samplings':samplings,
                   'resolutions':resolutions,
             'repeats':repeats,
            'snrs':snrs})
utg.save_as_pkl('','sim_varying_t2',data)

# 4. Simulating sampling

In [ ]:
bandwidth = 5000
dwell_time = 1/bandwidth
samplings = [32,64,80,128,200,256,512,600,750,900,1024,1200,1500,2048,3000,3500,4096,4500,5000]
repeats = 100
noise_base_level=0.01
b0=3
fieldoffset=170.4
noise_levels=np.linspace(noise_base_level,noise_base_level,1)

In [ ]:
temps_truth = np.zeros((len(noise_levels),repeats,len(samplings)))
temps_fit = np.zeros((len(noise_levels),repeats,len(samplings)))
pyr_lac_diffs_fit = np.zeros((len(noise_levels),repeats,len(samplings)))
pyr_lac_diffs_fit_ppm= np.zeros((len(noise_levels),repeats,len(samplings)))
pyr_lac_diffs_truth = np.zeros((len(noise_levels),repeats,len(samplings)))
resolutions = np.zeros((len(noise_levels),repeats,len(samplings)))
snrs = np.zeros((len(noise_levels),repeats,len(samplings)))

for sampling_index, sampling in tqdm(enumerate(samplings),total=len(samplings),desc='Sampling'):
    input_data = np.zeros((sampling,len(noise_levels),repeats,1,1,1),dtype=complex)
    for noise_level_index,noise_level in enumerate(noise_levels):
        for repeat_index in range(repeats):
            metab_fs = np.array([19.0,410.0,-313.0,273.0]) #Hz


            time = np.linspace(0, dwell_time * (sampling - 1), sampling)
            signal_temp =  uts.generate_fid(amplitude=[1.2, 0.85, 0.3, 0.3],
                                       T2_s=[0.05, 0.05, 0.05, 0.05],
                                       freq0_Hz=metab_fs,
                                       sampling_dt=dwell_time,
                                       npoints=sampling,
                                       noise_amplitude=noise_level,
                                       sum_fids=True) 
            input_data[:,noise_level_index,repeat_index,0,0,0]=signal_temp
            freq_range = np.fft.fftshift(np.fft.fftfreq(sampling,dwell_time))
            ppm_range = freq_range/(utg.get_gmr(nucleus="13c")*b0) + fieldoffset
            spectrum=np.fft.fftshift(np.fft.fft(signal_temp))
            pyr_peak_height = np.max(np.abs(spectrum))
            std = np.std(np.abs(spectrum[:50]))
            mean=np.mean(np.abs(spectrum[:50]))
            snr = (pyr_peak_height-mean)/std
            snrs[noise_level_index,repeat_index,sampling_index]=snr
    metabs = ['pyruvate', 'lactate', 'bicarbonate', 'pyruvatehydrate']
    niter = 2 # number of iterations:
    npoints  = 31# number of tested points per iteration:
    rep_fitting = 11
    # define fit parameters:
    fit_params = {}
    # define peak frequencies:
    fit_params["metabs"] = metabs
    fit_params["signal_domain"] = "temporal"
    fit_params["b0"] = b0
    fit_params["init_fit"] = True
    fit_params["coff"] = 0
    fit_params["zoomfactor"] = 1.5
    fit_params["metab_t2s"] = [0.05 for _ in metabs]
    fit_params["max_t2_s"] = [0.07, 0.07, 0.07, 0.07]
    fit_params["min_t2_s"] = [0.015, 0.03, 0.03, 0.03]
    fit_params["range_t2s_s"] = 0.05
    gmr = utg.get_gmr(nucleus="13c")
    fit_params["freq_range_Hz"] = freq_range
    fit_params["freq_range_ppm"] = fit_params["freq_range_Hz"]/(gmr*fit_params['b0']) + fieldoffset
    fit_params["range_freqs_Hz"] = 20
    fit_params["show_tqdm"] = True
    fit_params["metabs_freqs_ppm"] = [uts.get_metab_cs_ppm(metab=m) for m in metabs]
    # define peak frequencies:
    # TODO: implement maybe better finding of spectrum to get peak pos from
    spectrum = np.fft.fftshift(np.fft.fft(input_data[:,0,0,0,0,0]))
    max_peakindex = uts.find_npeaks(input_data=spectrum,
                                    npeaks=1,
                                    freq_range=fit_params['freq_range_Hz'],
                                    plot=False,
                                   sort_order=None)
    pyr_data =  fit_params["freq_range_ppm"][max_peakindex][0]
    pyr_lit = fit_params["metabs_freqs_ppm"][0]
    diff_pyr_data_lit = pyr_data - pyr_lit
    fit_params["metabs_freqs_ppm"] = [uts.get_metab_cs_ppm(metab=m)+diff_pyr_data_lit for m in metabs]

    fit_params["niter"] = niter # number of iterations:
    fit_params["npoints"] = npoints # number of tested points per iteration:
    fit_params["rep_fitting"] = rep_fitting # number of tested points per iteration:

    fit_params["fit_range_repetitions"] = 1
    fit_params["use_all_cores"]=True
    fit_params = utf.def_fit_params(fit_params=fit_params)
    fit_spectrums, fit_amps, fit_freqs, fit_t2s,  fit_stds = utf.fit_data_pseudo_inv(input_data=input_data,
                                                                      fit_params=fit_params, 
                                                                      dbplot=False,
                                                                      use_multiprocessing=True)

    # sorting back the results
    for noise_level_index,noise_level in enumerate(noise_levels):
        for repeat_index in range(repeats):
            metab_fs = np.array([19.0,410.0,-313.0,273.0]) #Hz

            freq_range = np.fft.fftshift(np.fft.fftfreq(sampling,dwell_time))
            ppm_range = freq_range/(utg.get_gmr(nucleus="13c")*b0) + fieldoffset
            spectral_resolution=bandwidth/sampling
            pyr_lac_diff_fit = fit_freqs[0,noise_level_index,repeat_index,0,0,0,1]-fit_freqs[0,noise_level_index,repeat_index,0,0,0,0]
            pyr_lac_diff_ppm_fit = uts.freq_Hz_to_ppm(pyr_lac_diff_fit,freq_range,ppm_range,ppm_centered_at_0=True)
            temp_fit,_=utf.temperature_from_frequency(pyr_lac_diff_ppm_fit,'5mM',True)

            pyr_lac_diff_truth=metab_fs[1]-metab_fs[0]
            pyr_lac_diff_ppm_truth=uts.freq_Hz_to_ppm(pyr_lac_diff_truth,freq_range,ppm_range,ppm_centered_at_0=True)
            temp_truth,_=utf.temperature_from_frequency(pyr_lac_diff_ppm_truth,'5mM',True)

            temps_truth[noise_level_index,repeat_index,sampling_index]=temp_truth
            temps_fit[noise_level_index,repeat_index,sampling_index]=temp_fit
            pyr_lac_diffs_fit[noise_level_index,repeat_index,sampling_index]=pyr_lac_diff_fit
            pyr_lac_diffs_fit_ppm[noise_level_index,repeat_index,sampling_index]=pyr_lac_diff_ppm_fit
            
            pyr_lac_diffs_truth[noise_level_index,repeat_index,sampling_index]=pyr_lac_diff_truth
            resolutions[noise_level_index,repeat_index,sampling_index]=spectral_resolution

In [ ]:
data = dict()
data.update({'temps_truth':temps_truth,
                   'temps_fit':temps_fit,
                   'pyr_lac_diffs_fit':pyr_lac_diffs_fit,
                   'pyr_lac_diffs_truth':pyr_lac_diffs_truth,
                   'noise_levels':noise_levels,
                   'samplings':samplings,
                   'resolutions':resolutions,
             'repeats':repeats,
            'snrs':snrs})
utg.save_as_pkl('','sim_sampling_rev1',data)